The use of sockets for transition state calculation using the Nudged Elastic Band (NEB) method can potentially half the calculation time. 

This is because after every iteration, there is an exchane of information between the calculator and the python code. 
With the use of sockets, the speed of this communication between the calculator and python/ase can be increased which helps in reducing the overall computational time. 

This is only useful for computers where the calculator use a different CPU than the python code during the simulation. ARCHER and Isambard are two machines where sockets can be used to get significant reduction in simulation time. HAWK will NOT have any advantage because it uses same CPUs for python as well as the calculator.

To use the sockets, we begin normally, like we do in a regular NEB calculation. We define the initial and final geometry and create a list of images.


In [ ]:
from ase.io import read, write
initial = read ('initial.traj')
final = read ('final.traj')
images = [initial.copy() for i in range(6)] + [final]

Next we interpolate the path to create the first guess pathway. 

To use the sockets effectively, we need to use the "Single Calculator". But this calculator does not have idpp interpolation built-in. 

So in oder to use the idpp interpolation, we use the regular NEB to generate the interpolated path. 

In [ ]:
from ase.neb import NEB
neb = NEB(images)
neb.interpolate('idpp')
# To use linear interpolation, users can replace idpp with linear. Or they can set the interpolation on the images later in the single calculator setting and skip this entire block of code.

Next we create a constrain to apply to all structures (if there is any):

In [ ]:
from ase.constraints import FixAtoms
constraint = None

Then we define the calculator. We do this by creating a function. 
We use the carmm library wich automatically returns the aims and sockets calculator. 

The library calls the calculator first, and then we set parameters on the calculator based on our preference.

Finally, the aims calculator is returned with the necessary sockets settings. 

In [ ]:
def my_calc():
	from carmm.run.aims_calculator import get_aims_and_sockets_calculator
	sockets_calc, fhi_calc = get_aims_and_sockets_calculator(dimensions=2)
	fhi_calc.set(xc='pbe', 
			k_grid='5 5 4',
			spin='none',
			relativistic=('atomic_zora','scalar'),
			compute_forces="true",
			final_forces_cleaned='true',
	return sockets_calc

Now we defined the NEB with the single calculator setting. And then finally run the optimizer within the calc function.

Running the optimizer within the "with" loop is essential for the calculator and python to effectively communicate over the socket.

In [ ]:
from ase.neb import SingleCalculatorNEB as NEB
with my_calc() as calc:
	neb = NEB(images)
	#neb.interpolate() #this can be used to specify linear method of interpolation. Note that idpp method cannot be specified here.
	for image in images:
		image.set_calculator(calc)
		image.set_constraint(constraint)

	# Relax the NEB path:
	from ase.optimize import FIRE
	minimizer = FIRE(neb, trajectory='neb.traj')
	minimizer.run(fmax=0.05)